In [1]:
import numpy as np
import pandas as pd

In [2]:
# rob-p5k
rootFolder = '~/Documents/Finance/'

In [3]:
# KAUWITB
rootFolder = '~/Data/Finance/'

In [4]:
# always target the newest export
Csv = 'SIMPLII_2023_01_10'

In [5]:
simpliiDf = pd.read_csv(rootFolder + 'simplii/' + Csv + '.csv', 
                       names = ['Date','Transaction_Details','Funds_Out','Funds_In'],
                       header=0,
                       dtype = {'Date':str, 'Transaction_Details':str, 'Funds_Out':np.double, 'Funds_In':np.double },
                       parse_dates=['Date']
                       )

simpliiDf['Funds_Out'] = simpliiDf['Funds_Out'].fillna(0)
simpliiDf['Funds_In'] = simpliiDf['Funds_In'].fillna(0)


Grab all the transactions from the money manager spreadsheet.

In [6]:
transactionsDf = pd.read_excel(rootFolder + 'money-manager-2 (2021).xlsx', 
                               sheet_name='Transactions', 
                               header=3)

In [7]:
# transaction Column names we want to generate
transactionColumns = ['Account', 'Date', 'Num', 'Payee', 'Memo', 'Tag', 'Category', 'Clr', 'PAYMENT', 'DEPOSIT']

Create a dataframe that maps Payee to Category and then save it to a csv file.

In [8]:
# Grab just the Payee and Category columns
pcDf = transactionsDf[['Payee','Category']]

In [9]:
# Some of these have leading spaces, so strip them out
pcDf = pcDf.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [10]:
# Add a Count column set to 1
pcDf['Count'] = 1

In [11]:
# Group by Payee and Category to get the count
pcDf = pcDf.groupby(['Payee','Category'])['Count'].count().reset_index()

In [12]:
# only keep those records with a count of 2 or higher
pcDf = pcDf[(pcDf.Count > 1)]

In [13]:
# Sort by Payee then Category
pcDf = pcDf.sort_values(by=['Payee', 'Category'], ascending=(True, True))

In [14]:
# Save this to a csv file
pcDf.to_csv(rootFolder + 'PayeeCategory.csv')

Merge simpliiDf with pcDf and create the dataframe you wish to append to the transactions tab.

In [15]:
# pd.merge(df1,df2, left_on='Courses', right_on='Courses', how='outer')
mergeDf = pd.merge(simpliiDf, pcDf, left_on='Transaction_Details', right_on='Payee', how='left')

In [16]:
# add missing columns
mergeDf['Account'] = 'Simplii Debit Card'
mergeDf['Num'] = ''
mergeDf['Memo'] = Csv + '.csv'
mergeDf['Tag'] = mergeDf.index + 1
mergeDf['Clr'] = ''
mergeDf['PAYMENT'] = mergeDf['Funds_Out']
mergeDf['DEPOSIT'] = mergeDf['Funds_In']

In [17]:
importDf = mergeDf[['Account',
 'Date',
 'Num',
 'Transaction_Details',
 'Memo',
 'Tag',
 'Category',
 'Clr',
 'PAYMENT',
 'DEPOSIT']]

In [18]:
simpliiImportDf = importDf.rename(columns = {'Transaction_Details':'Payee'})

In [19]:
# drop some rows we will not need on the import
# dataframe.drop(dataframe[dataframe['cost'] > 300].index)
simpliiImportDf.drop(simpliiImportDf[simpliiImportDf['Payee'].str.strip() == 'TRANSFER IN'].index, inplace=True)
simpliiImportDf.drop(simpliiImportDf[simpliiImportDf['Payee'].str.strip() == 'INTERNET BILL PAYMENT MASTERCARD PC FINANCIAL'].index, inplace=True)

In [20]:
# also drop bogus Canada Pension Plan records
simpliiImportDf.drop(simpliiImportDf[simpliiImportDf['Category'].str.strip() == 'Canada Pension Plan'].index, inplace=True)

In [21]:
simpliiImportDf['Category'] = simpliiImportDf['Category'].fillna('')

In [22]:
# Save this to a csv file
simpliiImportDf.to_csv(rootFolder + Csv + '_4Import.csv')